In [1]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

train_data = pd.read_pickle(open("tfidf/train_tfidf.pkl", "rb"))
train_X = list(train_data['TFIDF'])
train_y = train_data['Sentiment']
test_data = pd.read_pickle(open("tfidf/test_tfidf.pkl", "rb"))
test_X = list(test_data['TFIDF'])
test_y = test_data['Sentiment']

In [2]:
gnb = GaussianNB()
gnb.fit(train_X, train_y)
labels_predict = gnb.predict(test_X)
labels_predict

array(['positive', 'negative', 'negative', ..., 'negative', 'negative',
       'negative'], dtype='<U8')